In [ ]:
import pandas as pd
#para la conexion con la API opensubtitle.org
import xmlrpc.client

In [ ]:
def imdb_movies_by_year(year) :
    imdb_films = pd.read_csv('imdb_merged_movies.tsv', sep='\t') 
    imdb_films = imdb_films[imdb_films['startYear']==year]
    imdb_films = imdb_films.reset_index(drop=True)
    return imdb_films;

In [ ]:
#Me quedo con los ids sin el prefijo tt, me parece que la API de opensubtitles.org los necesita sin el prefijo
def clean(movie_id):
    clean = movie_id.replace("t","")
    clean = clean.replace(".zip","")
    return clean;

In [ ]:
#conexion a la API opensubtitle.org
url = 'https://api.opensubtitles.org/xml-rpc'
proxy = xmlrpc.client.ServerProxy(url)
username='ckruzy' #set your username here
password='t3xtM1n1ng2018' #set your password here
def token():
    token = proxy.LogIn(username,password ,'en','textMiningFinalProject')['token']
    return token;

In [ ]:
def openSubtitle_movies_info(lwr,parameters,dataframe=None):
    upr=lwr+500 if lwr+500<len(parameters) else len(parameters)
    data=proxy.SearchSubtitles(token(), parameters[lwr:upr])
    if dataframe is not None:
        dataframe=dataframe.append(pd.DataFrame(data['data']))
    else:
        dataframe=pd.DataFrame(data['data'])
    return dataframe;

In [ ]:
def openSubtitle_movies(imdb_movies):
    parameters=[]
    for x in range(0,len(imdb_movies)):
        parameters.append({'imdbid':clean(imdb_movies.tconst[x]),'sublanguageid':'eng'})
    lwr=0
    while (lwr<len(parameters)):
        movies=openSubtitle_movies_info(lwr,parameters) if lwr==0 else openSubtitle_movies_info(lwr,parameters,movies)
        lwr=lwr+500
    try:
        movies=movies.drop_duplicates(subset='IDMovieImdb', keep='first', inplace=False)
    except KeyError:
        print("KeyError ")
        print(len(movies))
    movies = movies.reset_index(drop=True)
    return movies;

In [ ]:
#opensubtitles.org tiene un límite de descarga de 200 subtítulos por día para las cuentas gratuitas
def download_Subtitles(movies,lwr):
    upr=lwr+20

    while (lwr<=len(movies)):
        subtitleszips=proxy.DownloadSubtitles(token(),movies['IDSubtitleFile'][lwr:upr].tolist())
        try:
            saveSubtitles(subtitleszips, movies)
        except TypeError:
            print("Limit exceed ... lwr="+str(lwr))
            return lwr
        lwr=lwr+20
        upr=upr+20 if upr+20 < len(movies) else len(movies)
    
    return lwr;

In [ ]:
import base64
def saveSubtitles(zips,movies):
    len(zips['data'])
    for subtitle in zips['data']:
        file=base64.b64decode(subtitle['data'])
        idsubtitlefile = subtitle['idsubtitlefile']
        idimdb = movies[movies['IDSubtitleFile']==idsubtitlefile]['IDMovieImdb'].reset_index(drop=True)[0]
        with open('tt'+idimdb+'.zip','wb') as f:
            f.write(file)
            f.close()

In [ ]:
from os import listdir
def drop_downloaded(movies,year):
    folder_path = "./subtitles"+str(year)+"/" 
    file_list = listdir(folder_path)
    file_list = [x.replace(".zip","") for x in file_list]

    new_movies=movies[~movies.tconst.isin(file_list)]
    new_movies=new_movies.reset_index(drop=True)
    return new_movies;

In [ ]:
year=2018.0
movies=imdb_movies_by_year(year) #busca los títulos del año indicado
print(len(movies))
movies=drop_downloaded(movies,int(year)) #elimina los ids de aquellos para los que ya descargo los subtitulos
len(movies)

In [ ]:
openSub_movies=openSubtitle_movies(movies)#Busca información de los subtitulos en opensubtiles.org
print(len(openSub_movies))

In [ ]:
cant=download_Subtitles(openSub_movies,0) #Descarga los archivos de los subtitulos de opensubtitles.org
print(cant)